## Código de preparación de datos...

Este pequeño código prepara los datos para ser manipulados y ser usados por otro código de propagación de muones en roca.. La idea es que tenemos dos archivos de datos. El primero es la salida de muones del volcán machin en el punto P1 de observación, el otro archivo de datos es la distancia recorrida por los muones en roca, calculado a partir de un código en matlab externo. La idea es manipular el primero de los archivos para obtener la energía total a partir de la consideración de los momentos en las direcciones $x$, $y$, y $z$. Lo segundo es teniendo la energía vamos a organizar los datos de tal manera de calcular el flujo, es decir, sumar todos los muones que llegan en la misma dirección para tener el número de muones que entran por esa determinada dirección, con esto tendremos una data de flujo por dirección.
Lo siguiente es combinar el archivo de datos de distancias recorridas y le ponga a cada partícula entrante con energía determinada E, su correspondiente distancia recorrida...

In [1]:
# Cargando todas las librerías que necesitamos
import numpy as np
import math
import pandas as pd
from scipy.stats import itemfreq
import matplotlib.pyplot as plt
#ff_int=lambda x: "%i" % x
#ff_dec=lambda x: "%.1f" % x
#np.set_printoptions(formatter={'float_kind':ff_int})
!ls

distp1_machin.out		   salidaEab_prueba60.txt
flux_buca_filter.ipynb		   salidaEab_prueba70.txt
flux_filter_machinp1.ipynb	   salidaEab_prueba80.txt
muones_buca_1h.out		   salidaEab_prueba90.txt
muonesBucara.out		   salidaGauss_prueba100.txt
muones_to_propagate.out		   salidaGauss_prueba30.txt
muones_to_propagate_prueba100.out  salidaGauss_prueba40.txt
muones_to_propagate_prueba30.out   salidaGauss_prueba50.txt
muones_to_propagate_prueba40.out   salidaGauss_prueba60.txt
muones_to_propagate_prueba50.out   salidaGauss_prueba70.txt
muones_to_propagate_prueba60.out   salidaGauss_prueba80.txt
muones_to_propagate_prueba70.out   salidaGauss_prueba90.txt
muones_to_propagate_prueba80.out   totalrealMachin.out
muones_to_propagate_prueba90.out   totalrealMachin_tpcount_ordenados.out
MuonStoppingPower2.txt		   totalrealMachin_tpdE_ordenados.out
salidaEab_prueba100.txt		   totalrealMachin_tpE_ordenados.out
salidaEab_prueba30.txt		   tpE_muones_buca.out
salidaEab_prueba40.txt		   tracking_Eab.py
sali

Llamamos al primer archivo de datos originales que vienen de Corsika...

In [2]:
df1=pd.read_csv('totalrealMachin.out',sep=' ',names=['px','py','pz','p'])
df1.head(5)

,px,py,pz,p
0,0.243761,0.212928,2.059700,2
1,12.152900,-2.391790,2.702590,12
2,-0.108859,0.137819,0.688568,0
3,1.770120,0.317097,2.728950,3
4,0.321635,-0.216749,0.646460,0


Acá ahora tomamos losmomentos y calculamos la energía total de cada muon que llega al volcán... En este caso 
\begin{equation}
\theta=\arccos\frac{p_z}{p}=\arccos\frac{p_z}{\sqrt{p_x^2+p_y^2+p_z^2}}, \ \ \phi=\arctan\frac{p_y}{p_x}
\end{equation}

In [3]:
px,py,pz=df1['px'],df1['py'],df1['pz']
pt=np.sqrt(px**2+py**2+pz**2)
m_rest=0.1057 #unit: GeV/c**2
Et=np.sqrt(pt**2+m_rest**2)
thetat=np.degrees(np.arccos(pz/pt))
at2=np.arctan2(py,px)
phit=np.degrees((2*np.pi+at2)*(at2<0)+(at2)*(at2>0))

Ahora tenemos un archivo de ángulos de incidencia vs. energía con la que llegan. Guardamos los datos nuevos con el formato requerido... Cargamos la nueva dataframe y ordenamos por ángulos.

In [4]:
data=np.c_[phit,thetat,Et]
data=data[data[:,1].argsort()] # Ordena de acuerdo a angulo theta
# Obtenemos solo los que están en el rango 117<phi<147 y 66<theta<84
data=data[(data[:,0]>=117) & (data[:,0]<=148) & (data[:,1]>=66) & (data[:,1]<=85)]
data=np.savetxt('totalrealMachin_tpE_ordenados.out',data,delimiter=' ',fmt='%i %i %2f')
df2=pd.read_csv('totalrealMachin_tpE_ordenados.out',sep=' ',names=['phi','theta','E'])
df2=df2.sort_values(['phi','theta','E']).reset_index(drop=True)
df2.head(5)
df2.to_csv('totalrealMachin_tpE_ordenados.out',sep=' ',index=False)
df2

,phi,theta,E
0,117,66,0.163945
1,117,66,0.166845
2,117,66,0.174983
3,117,66,0.244354
4,117,66,0.266079
5,117,66,0.285337
6,117,66,0.288479
7,117,66,0.382494
8,117,66,0.399445
9,117,66,0.434121


Ahora agrupamos todos los muones que tengan la misma dirección de incidencia y con esto tenemos el flujo por ángulo.

In [5]:
df3=df2.groupby(['phi','theta'])['E'].count().reset_index(name='count')
df3.to_csv('totalrealMachin_tpcount_ordenados.out',sep=' ',index=False)
df3

,phi,theta,count
0,117,66,510
1,117,67,452
2,117,68,410
3,117,69,402
4,117,70,375
5,117,71,344
6,117,72,275
7,117,73,252
8,117,74,221
9,117,75,195


Cargamos el archivo de las distancias..

In [6]:
df4=pd.read_csv('distp1_machin.out',sep='\t',names=['phi','theta','d'],skiprows=3)
df4

,phi,theta,d
0,117,66,0
1,118,66,0
2,119,66,0
3,120,66,0
4,121,66,0
5,122,66,0
6,123,66,0
7,124,66,0
8,125,66,0
9,126,66,0


Y ahora combinamos las energías para cada muon en cada dirección de incidencia con las distancias recorridas en esas mismas incidencias...

In [7]:
m = df4.merge(df2, on=['phi', 'theta'], how='outer')
m.to_csv('totalrealMachin_tpdE_ordenados.out',sep=' ',index=False)
m

,phi,theta,d,E
0,117,66,0,0.163945
1,117,66,0,0.166845
2,117,66,0,0.174983
3,117,66,0,0.244354
4,117,66,0,0.266079
5,117,66,0,0.285337
6,117,66,0,0.288479
7,117,66,0,0.382494
8,117,66,0,0.399445
9,117,66,0,0.434121


Preparando el archivo final de entrada para el código de propagación...

In [8]:
muones_to_propagate=np.loadtxt('totalrealMachin_tpdE_ordenados.out',delimiter=' ',skiprows=2)
#print muones_to_propagate

In [9]:
dis=muones_to_propagate[:,2]*100 # Convert to cm, originally in m, code and tables use cm
ener=muones_to_propagate[:,3]*1000 # Convert to MeV, originally in GeV, code and tables use MeV
muones_to_propagate=np.c_[dis,ener]
#print muones_to_propagate
np.savetxt('muones_to_propagate.out',muones_to_propagate,delimiter=' ',fmt='%i %2f')

Para una pequeña prueba vamos a quedarnos solo con los indices múltiplos de 30...

In [10]:
muones_to_propagate=muones_to_propagate[(muones_to_propagate[:,0]>0)]
muones_to_propagate_prueba=muones_to_propagate[1::100]
#muones_to_propagate_prueba=muones_to_propagate_prueba[::2]
#muones_to_propagate_prueba=muones_to_propagate_prueba[::2]
np.savetxt('muones_to_propagate_prueba100.out',muones_to_propagate_prueba,delimiter=' ',fmt='%i %2f')

In [11]:
print np.shape(muones_to_propagate_prueba)

(1110, 2)
